In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy.io import loadmat
from scipy.optimize import minimize

data = loadmat('./Data/ex3data1.mat')

X = data['X']
y = data['y']
print('X Shape : ', X.shape)
print('Y Shape : ', y.shape)

encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
print('Y One-Hot Shape : ',y_onehot.shape)

X Shape :  (5000, 400)
Y Shape :  (5000, 1)
Y One-Hot Shape :  (5000, 10)


c:\python\venv\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [2]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def forward_propagation(X, theta_1, theta_2):
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1*theta_1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2*theta_2.T
    h = sigmoid(z3)

    return a1, z2, a2, z3, h

def cost_fn(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    theta_1 = np.matrix(np.reshape(params[:hidden_size * (input_size+1)], (hidden_size, (input_size+1))))
    theta_2 = np.matrix(np.reshape(params[hidden_size * (input_size+1):], (num_labels, (hidden_size+1))))

    a1, z2, a2, z3, h = forward_propagation(X, theta_1, theta_2)

    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i, :]))
        second_term = np.multiply((1-y[i, :]), np.log(1-h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J/m
    return J

In [3]:
input_size = 400
hidden_size = 25
num_labels = 10
learning_rate = 0.1

params = (np.random.random(size=hidden_size * (input_size + 1)+num_labels*(hidden_size+1))-0.5)*0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

theta_1 = np.matrix(np.reshape(params[:hidden_size*(input_size+1)], (hidden_size, (input_size+1))))
theta_2 = np.matrix(np.reshape(params[hidden_size * (input_size+1):], (num_labels, (hidden_size+1))))

a1, z2, a2, z3, h = forward_propagation(X, theta_1, theta_2)
print('cost : ', cost_fn(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate))

cost :  7.073733164898658


In [4]:
def sigmoidGradient(z):
    return np.multiply(sigmoid(z), (1-sigmoid(z)))

def back_propagation(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    theta_1 = np.matrix(np.reshape(params[:hidden_size * (input_size+1)], (hidden_size, (input_size+1))))
    theta_2 = np.matrix(np.reshape(params[hidden_size*(input_size+1):], (num_labels, (hidden_size+1))))

    a1, z2, a2, z3, h = forward_propagation(X, theta_1, theta_2)

    J = 0
    delta1 = np.zeros(theta_1.shape)
    delta2 = np.zeros(theta_2.shape)

    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i, :]))
        second_term = np.multiply((1-y[i, :]), np.log(1-h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J/m

    J += (float(learning_rate)/(2*m))*(np.sum(np.power(theta_1[:, 1:],2))+np.sum(np.power(theta_2[:, 1:], 2)))

    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        ht = h[t,:]
        yt = y[t,:]

        d3t = ht - yt

        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta_2.T * d3t.T).T, sigmoidGradient(z2t))

        delta1 = delta1 + (d2t[:, 1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t

    delta1 = delta1 / m
    delta2 = delta2 / m

    delta1[:, 1:] = delta1[:, 1:] + (theta_1[:, 1:] * learning_rate) / m
    delta2[:, 1:] = delta2[:, 1:] + (theta_2[:, 1:] * learning_rate) / m

    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

In [5]:
J, grad = back_propagation(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

print('----- Training -----')
f_min = minimize(fun=back_propagation, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate), method='TNC', jac=True, options={"maxiter":100})

X = np.matrix(X)
theta_1 = np.matrix(np.reshape(f_min.x[:hidden_size * (input_size+1)], (hidden_size, (input_size+1))))
theta_2 = np.matrix(np.reshape(f_min.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size+1))))

a1, z2, a2, z3, h = forward_propagation(X, theta_1, theta_2)
y_pred = np.array(np.argmax(h, axis=1)+1)

correct = [1 if a == b else 0 for (a,b) in zip(y_pred, y)]
acc = (sum(map(int, correct))/float(len(correct)))
print('Accuracy : {0} %'.format(acc*100))

----- Training -----
Accuracy : 96.98 %
